In [3]:
# Data Analysis and Modeling
import pandas as pd
from sklearn.model_selection import train_test_split

# TF-IDF Analysis packages
from sklearn.feature_extraction.text import TfidfVectorizer

# Packages for PostgreSQL Import
import psycopg2

In [4]:
# Ideally I'll move this into the project config.py file
# Otherwise for now I have to just manually assign . . .

dbname = "freelance_db"
username = "Metaverse"
pswd = "Arcifice91"

In [5]:
# Connect to Data (from: scraping_data.ipynb)

con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# Extract freelance_db as fl_table, don't bring Punjab obs
sql_query = """SELECT * FROM analysis_table;"""
analysis_dt = pd.read_sql_query(sql_query, con)
analysis_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               948 non-null    int64  
 1   invoices_per_month  948 non-null    float64
 2   hourly_rate         948 non-null    int64  
 3   earnings_pst_yr     948 non-null    object 
 4   earnings_pr_month   948 non-null    float64
 5   Midwest             948 non-null    int64  
 6   Northeast           948 non-null    int64  
 7   South               948 non-null    int64  
 8   Other               948 non-null    int64  
 9   West                948 non-null    int64  
 10  less_five_skills    948 non-null    int64  
 11  bio_length          948 non-null    int64  
 12  bio_word_count      948 non-null    int64  
 13  avg_word_length     948 non-null    float64
 14  num_stop            948 non-null    int64  
 15  bio_processed       948 non-null    object 
dtypes: float

### Trying tf-idf

1. I think this only works if I have a classifier
2. Could build something that tells someone if they have a top performing bio?
    - Weak because a big part of this is the LOOK of the bio.
    - Always could provide URL references.

In [13]:
vectorizer = TfidfVectorizer()
analyze = vectorizer.build_analyzer()

analyze(analysis_dt.bio_processed[0])
analyze(analysis_dt.bio_processed[1])



['top',
 'guru',
 'assistants',
 'team',
 '100',
 'professionals',
 'experts',
 'skills',
 'collaborating',
 'together',
 'common',
 'vision',
 'unity',
 'build',
 'longterm',
 'business',
 'relationships',
 'helping',
 'clients',
 'leverage',
 'technology',
 'scale',
 'business',
 'next',
 'level',
 'who',
 'we',
 'are',
 'physical',
 'presence',
 'us',
 'india',
 'canada',
 'singapore',
 'malaysia',
 'uae',
 'iso',
 'certified',
 'microsoft',
 'partner',
 'developer',
 'program',
 'registered',
 'apple',
 'google',
 'fastest',
 'growing',
 'team',
 'freelancersfew',
 'facts',
 'about',
 'us',
 'million',
 'lines',
 'code',
 'written',
 '100',
 'client',
 'appreciation',
 'emails',
 'years',
 'counting',
 '100',
 'projects',
 'delivered',
 'verifiable',
 'client',
 'referencesservices',
 'offered',
 'website',
 'web',
 'application',
 'development',
 'mobile',
 'application',
 'development',
 'enterprise',
 'services',
 'startup',
 'servicesengagement',
 'models',
 'offshore',
 'onsit